# Running VAAS on CPU, CUDA, and Apple Silicon (MPS)

VAAS is designed to run on different hardware backends depending on
what is available in the execution environment.

This notebook explains:
- how device selection works in VAAS,
- how to run inference on CPU, CUDA GPUs, or Apple Silicon (MPS),
- and what practical differences to expect between these options.

## Installation

Install VAAS if it is not already available in the environment.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO

## Loading an Example Image

We use a single image throughout this notebook to keep comparisons
between devices straightforward.

In [ ]:
image_url = (
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/"
    "examples/images/COCO_DF_C110B00000_00539519.jpg"
)

response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

image


## How Device Selection Works

VAAS relies on PyTorch for model execution.
The device is specified when initializing the VAAS pipeline.

Common options are:
- "cpu"   : runs on the system CPU
- "cuda"  : runs on an NVIDIA GPU (if available)
- "mps"   : runs on Apple Silicon via Metal Performance Shaders

If a requested device is not available, PyTorch will raise an error.
VAAS does not silently fall back to another device.

---

## Running VAAS on CPU

CPU execution is the most portable option.
It works in all environments, including Colab and local machines
without dedicated GPUs.


In [ ]:
pipeline_cpu = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",
    alpha=0.5,
)

result_cpu = pipeline_cpu(image)

print("CPU S_H:", result_cpu["S_H"])

## Running VAAS on CUDA (NVIDIA GPU)

If an NVIDIA GPU is available and PyTorch is installed with CUDA support,
VAAS can run on the GPU by setting the device to "cuda".

In Colab, this requires enabling a GPU runtime.


In [ ]:
# This cell will work only if a CUDA-enabled GPU is available.
# If not, it can be safely skipped.

pipeline_cuda = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cuda",
    alpha=0.5,
)

result_cuda = pipeline_cuda(image)

print("CUDA S_H:", result_cuda["S_H"])

## Running VAAS on Apple Silicon (MPS)

On Apple Silicon machines, PyTorch provides MPS support.
This allows VAAS to run on the integrated GPU.

MPS support depends on:
- macOS version,
- PyTorch build,
- and hardware generation.


In [ ]:
# This cell is intended for Apple Silicon environments.
# It will only work if PyTorch reports MPS availability.

pipeline_mps = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="mps",
    alpha=0.5,
)

result_mps = pipeline_mps(image)

print("MPS S_H:", result_mps["S_H"])

## Comparing Results Across Devices

In most cases, the anomaly scores produced by VAAS should be
numerically very close across devices.

Minor floating-point differences may occur due to:
- hardware-specific kernels,
- different numerical precision paths,
- backend implementation details.

These differences are generally negligible for practical use.


## Performance Considerations

- CPU execution is slower but maximally portable.
- CUDA execution provides the best performance for batch or repeated inference.
- MPS provides a practical middle ground on Apple Silicon systems.

For one-off image inspection, CPU execution is often sufficient.
For larger workloads, GPU acceleration is recommended.


## Summary

In this notebook, we:
- examined how VAAS selects execution devices,
- ran inference on CPU, CUDA, and MPS backends,
- and discussed performance and portability trade-offs.

The next notebook will focus on loading VAAS models from Hugging Face
and understanding how model versioning and checkpoints are handled.

check it out [06_loading_vaas_models_from_huggingface.ipynb](https://colab.research.google.com/drive/16X5S_aarUKGktMYlW2bo2Fp4p5VX5p85?usp=sharing)
